In [1]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.formula.api import ols

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Q1

In [19]:
a_hat = -0.002
b_hat = 0.025
SSE = 0.022
n = 100
sigma_squared = SSE / (n - 2)

# Given XtX matrix and its inverse
XtX = np.array([[100, 65.24], [65.24, 44.64]])
XtX_inv = np.linalg.inv(XtX)

# Calculating the standard errors for a_hat and b_hat
# The variances are on the diagonal of XtX_inv and need to be multiplied by sigma_squared
var_a = XtX_inv[0, 0] * sigma_squared
var_b = XtX_inv[1, 1] * sigma_squared

SE_a_hat = np.sqrt(var_a)
SE_b_hat = np.sqrt(var_b)

# Calculating the t-statistics for a_hat and b_hat
t_stat_a = a_hat / SE_a_hat
t_stat_b = b_hat / SE_b_hat

SE_a_hat, SE_b_hat, t_stat_a, t_stat_b

(0.006945408963833088,
 0.010395268955405929,
 -0.28796000500685043,
 2.404940180696245)

# Q2

In [2]:
data = pd.read_csv('TestA_data.csv')
data

,Date,F90d,Spot
0,1991-01,1.166377,1.156013
1,1991-02,1.164625,1.154885
2,1991-03,1.166150,1.157150
3,1991-04,1.162823,1.153508
4,1991-05,1.158177,1.149851
...,...,...,...
311,2016-12,1.331350,1.334631
312,2017-01,1.317795,1.321098
313,2017-02,1.309963,1.309945
314,2017-03,1.337122,1.338855


In [3]:
Spot_3 = data['Spot'][3:].copy()
Spot_3 = Spot_3.reset_index(drop=True)
Spot_3

0      1.153508
1      1.149851
2      1.143893
3      1.149313
4      1.145142
         ...   
308    1.334631
309    1.321098
310    1.309945
311    1.338855
312    1.344281
Name: Spot, Length: 313, dtype: float64

In [4]:
Spot_0 = data['Spot'][:-3].copy()
Spot_0 = Spot_0.reset_index(drop=True)
Spot_0

0      1.156013
1      1.154885
2      1.157150
3      1.153508
4      1.149851
         ...   
308    1.310429
309    1.324224
310    1.345730
311    1.334631
312    1.321098
Name: Spot, Length: 313, dtype: float64

In [5]:
Fwd = data[:-3]['F90d'].copy()
Fwd = Fwd.reset_index(drop=True)
Fwd

0      1.166377
1      1.164625
2      1.166150
3      1.162823
4      1.158177
         ...   
308    1.310181
309    1.324260
310    1.342590
311    1.331350
312    1.317795
Name: F90d, Length: 313, dtype: float64

In [6]:
SpotC = Spot_3 - Spot_0
SpotC

0     -0.002506
1     -0.005034
2     -0.013258
3     -0.004195
4     -0.004709
         ...   
308    0.024202
309   -0.003126
310   -0.035785
311    0.004224
312    0.023183
Name: Spot, Length: 313, dtype: float64

In [7]:
FwdPre = Fwd - Spot_0
FwdPre

0      0.010364
1      0.009740
2      0.009000
3      0.009315
4      0.008326
         ...   
308   -0.000248
309    0.000036
310   -0.003140
311   -0.003281
312   -0.003303
Length: 313, dtype: float64

# Q2

In [8]:
result = adfuller(Spot_3)

# Extract ADF test statistic and p-value
adf_statistic = result[0]
p_value = result[1]

print("ADF Test Statistic:", adf_statistic)
print("p-value:", p_value)

ADF Test Statistic: -1.535563389286989
p-value: 0.5158510690843667


In [9]:
result = adfuller(Fwd)

# Extract ADF test statistic and p-value
adf_statistic = result[0]
p_value = result[1]

print("ADF Test Statistic:", adf_statistic)
print("p-value:", p_value)

ADF Test Statistic: -1.5452950827240772
p-value: 0.5109974997320496


In [10]:
result = adfuller(SpotC)

# Extract ADF test statistic and p-value
adf_statistic = result[0]
p_value = result[1]

print("ADF Test Statistic:", adf_statistic)
print("p-value:", p_value)

ADF Test Statistic: -3.3892431768542064
p-value: 0.011331082640266337


In [11]:
result = adfuller(FwdPre)

# Extract ADF test statistic and p-value
adf_statistic = result[0]
p_value = result[1]

print("ADF Test Statistic:", adf_statistic)
print("p-value:", p_value)

ADF Test Statistic: -3.067364128985892
p-value: 0.02906456949236092


# Q3

In [12]:
df1 = pd.DataFrame({'Spot_3':Spot_3, 'Fwd':Fwd})

In [13]:
df2 = pd.DataFrame({'SpotC':SpotC, 'FwdPre':FwdPre})

In [14]:
result_1 = coint_johansen(df1, 0, 3)
result_2 = coint_johansen(df2, 0, 3)

# Extract the number of cointegrating vectors (r)
r_1 = result_1.lr1[0]  # Trace statistic
r_2 = result_2.lr1[0]

print("Number of cointegrating vectors between Spot_3 and Fwd:", r_1)
print("Number of cointegrating vectors between SpotC and FwdPre:", r_2)


Number of cointegrating vectors between Spot_3 and Fwd: 34.045886371157884
Number of cointegrating vectors between SpotC and FwdPre: 40.41793070516116


In [15]:
result_1 = coint_johansen(df1, 0, 3)
result_2 = coint_johansen(df2, 0, 3)

critical_values_trace_1 = result_1.cvt[:, 0]  # Critical values for the trace statistic
critical_values_trace_2 = result_2.cvt[:, 0]
critical_values_max_eigen_1 = result_1.cvm[:, 0]  # Critical values for the max eigenvalue statistic
critical_values_max_eigen_2 = result_2.cvm[:, 0]

# Extract the test statistics
trace_statistic_1 = result_1.lr1[0]  # Trace statistic
trace_statistic_2 = result_2.lr1[0]
max_eigen_statistic_1 = result_1.lr2[0]  # Max Eigenvalue statistic
max_eigen_statistic_2 = result_2.lr2[0]

# Determine the number of cointegrating vectors based on the significance level
r_trace_1 = sum(trace_statistic_1 > critical_values_trace_1)
r_trace_2 = sum(trace_statistic_2 > critical_values_trace_2)
r_max_eigen_1 = sum(max_eigen_statistic_1 > critical_values_max_eigen_1)
r_max_eigen_2 = sum(max_eigen_statistic_2 > critical_values_max_eigen_2)

print("Trace Statistic:")
print("Number of cointegrating vectors between Spot_3 and Fwd:", r_trace_1)
print("Number of cointegrating vectors between SpotC and FwdPre:", r_trace_2)
print("\nMax Eigenvalue Statistic:")
print("Number of cointegrating vectors between Spot_3 and Fwd:", r_max_eigen_1)
print("Number of cointegrating vectors between SpotC and FwdPre:", r_max_eigen_2)

Trace Statistic:
Number of cointegrating vectors between Spot_3 and Fwd: 2
Number of cointegrating vectors between SpotC and FwdPre: 2

Max Eigenvalue Statistic:
Number of cointegrating vectors between Spot_3 and Fwd: 2
Number of cointegrating vectors between SpotC and FwdPre: 2


# Q4

In [16]:
df4 = pd.DataFrame({'Spot_3':Spot_3, 'Fwd':Fwd})

# Fit the linear regression model
model = ols('Spot_3 ~ Fwd', data=df4).fit()

# Print the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Spot_3   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     4934.
Date:                Mon, 18 Mar 2024   Prob (F-statistic):          7.34e-193
Time:                        12:52:41   Log-Likelihood:                 535.90
No. Observations:                 313   AIC:                            -1068.
Df Residuals:                     311   BIC:                            -1060.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0368      0.018      2.085      0.0

# Q6

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp



# Calculate errors (residuals)
errors = Spot_3 - Fwd

# Perform one-sample t-test
t_statistic, p_value = ttest_1samp(errors, popmean=0)

# Print the test results
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)


T-Statistic: 0.21799388871143324
P-Value: 0.8275763788907563


In [18]:
from scipy.stats import ttest_1samp
errors = model.resid

# Perform one-sample t-test
t_statistic, p_value = ttest_1samp(errors, popmean=0)

# Print the test results
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

T-Statistic: -2.453286224465143e-14
P-Value: 0.9999999999999805
